<a href="https://colab.research.google.com/github/alejovillores/tp2_spark_villores_alejo/blob/main/ejercicio_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejercicio 14
El Top 5 de contenidos que tienen la mayor cantidad de redirecciones que apuntan a
ellos.

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 59.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ae6159fa0d516ccc93b01718c78c0507c0c8c9cc6ba7fa9cc2b71dbe9997c0d0
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bioni

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/2C2021/Datos/Trabajo Practico 2/redirect_list.csv', header=True, inferSchema=True)
rdd_redirect = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Ya con todo listo e instalado, procedemos a realizar el ejercicio

In [ ]:
rdd_redirect.take(5)

[Row(rd_from=54, rd_namespace=0, rd_title='Apiales', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=57, rd_namespace=0, rd_title='Atmósfera', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=58, rd_namespace=0, rd_title='Archaea', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=82, rd_namespace=0, rd_title='Agujero_de_gusano', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=108, rd_namespace=0, rd_title='Stipa', rd_interwiki=None, rd_fragment=None)]

In [ ]:
help(rdd_redirect)

Limpio aquellos rd_from: ID del contenido que redirige

In [ ]:
rdd_redirect_cleaned = rdd_redirect.filter(lambda x: x.rd_from != None)

In [ ]:
grouped_by_title = rdd_redirect_cleaned.map(lambda x: (x.rd_title, 1)).reduceByKey(lambda x,y: x+y)

Buscamos los contenidos que mayor redirecciones poseen utilizando la accion takeOrdered

```
   |  takeOrdered(self, num, key=None)
   |      Get the N elements from an RDD ordered in ascending order or as
   |      specified by the optional key function
```

In [ ]:
grouped_by_title.takeOrdered(5, lambda x: -x[1])

[('Artículo_futuro', 1161),
 ('IV_milenio', 1001),
 ('V_milenio', 998),
 ('Estaciones_de_Metrobús_de_la_Ciudad_de_México', 396),
 ('163.117.0.0', 310)]